In [1]:
import cx_Oracle
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk

In [2]:
connection = cx_Oracle.connect("etl_dpl/oracle123@192.168.0.57:1521/TRNG")
cursor = connection.cursor()

In [3]:
cursor.execute("""
    SELECT C.CUSTOMER_ID,C.CUSTOMER_NAME,O.ORDERNUMBER,O.BUILDSTARTDATE,O.CHANNEL,O.CURRENCY_CODE,
    O.WAREHOUSE_CODE,O.DELIVERY_REGION_ID,O.PRODUCT_ID,O.ORDER_QUANTITY,O.UNIT_PRICE,O.LINE_TOTAL,O.TOTAL_UNIT_COST,
    P.PRODUCT_NAME,R.CITY,R.COUNTRY
    FROM CUSTOMERS C 
    LEFT JOIN ORDERS O 
    ON C.CUSTOMER_ID = O.CUSTOMER_ID
    LEFT JOIN PRODUCTS P
    ON C.CUSTOMER_ID = P.PRODUCT_ID
    LEFT JOIN REGIONS R
    ON O.DELIVERY_REGION_ID = R.REGION_ID
""")
result = cursor.fetchall()
columns = [col[0] for col in cursor.description]
data = [dict(zip(columns, row)) for row in result]

In [4]:
df = pd.DataFrame(data)

In [5]:
columns_to_drop = ['CUSTOMER_ID', 'CUSTOMER_NAME', 'ORDERNUMBER', 'CURRENCY_CODE', 'WAREHOUSE_CODE']
df.drop(columns=columns_to_drop, inplace=True)

In [6]:
df['BUILDSTARTDATE'] = pd.to_datetime(df['BUILDSTARTDATE'])

In [7]:
df['MONTH'] = df['BUILDSTARTDATE'].dt.month
df['YEAR'] = df['BUILDSTARTDATE'].dt.year

In [8]:
df.drop(columns=['BUILDSTARTDATE', 'LINE_TOTAL', 'TOTAL_UNIT_COST','PRODUCT_NAME','CHANNEL','CITY'], inplace=True)

In [9]:
df

,DELIVERY_REGION_ID,PRODUCT_ID,ORDER_QUANTITY,UNIT_PRICE,COUNTRY,MONTH,YEAR
0,5,6,11,5045.1,Australia,9,2015
1,24,22,12,3530.9,Australia,10,2015
2,82,8,10,1118.9,Australia,10,2015
3,35,1,6,1038.5,Australia,10,2015
4,79,1,8,1862.6,Australia,10,2015
...,...,...,...,...,...,...,...
4290,24,14,11,3631.4,Australia,12,2018
4291,61,25,10,1105.5,Australia,12,2018
4292,43,12,7,174.2,Australia,12,2018
4293,31,16,8,268.0,Australia,12,2018


In [10]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['COUNTRY'] = label_encoder.fit_transform(df['COUNTRY'])
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print(mapping)

{'Australia': 0}


In [11]:
df

,DELIVERY_REGION_ID,PRODUCT_ID,ORDER_QUANTITY,UNIT_PRICE,COUNTRY,MONTH,YEAR
0,5,6,11,5045.1,0,9,2015
1,24,22,12,3530.9,0,10,2015
2,82,8,10,1118.9,0,10,2015
3,35,1,6,1038.5,0,10,2015
4,79,1,8,1862.6,0,10,2015
...,...,...,...,...,...,...,...
4290,24,14,11,3631.4,0,12,2018
4291,61,25,10,1105.5,0,12,2018
4292,43,12,7,174.2,0,12,2018
4293,31,16,8,268.0,0,12,2018


In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
scaling=MinMaxScaler()

In [14]:
df['ORDER_QUANTITY'] = scaling.fit_transform(df[['ORDER_QUANTITY']])

In [15]:
X = df[['DELIVERY_REGION_ID','PRODUCT_ID', 'UNIT_PRICE','MONTH','YEAR','COUNTRY']]
y = df['ORDER_QUANTITY']

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train,y_train)

RandomForestRegressor(random_state=42)

In [18]:
predictions = rf_model.predict(X_test)

In [20]:
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")


Mean Squared Error: 0.1115187094628305
Root Mean Squared Error: 0.33394417117660624


In [21]:
 df_test = pd.read_excel(r'C:\Users\USER\Desktop\pp.xlsx')

In [22]:
df_test.head()

,CHANNEL,DELIVERY_REGION_ID,PRODUCT_ID,UNIT_PRICE,CITY,COUNTRY,MONTH,YEAR
0,Wholesale,5,6,5045.1,Broken Hill,Australia,1,2019
1,Export,24,22,3530.9,Wollongong,Australia,1,2019
2,Distributor,82,8,1118.9,Fermantle,Australia,1,2019


In [23]:
df_test.drop(columns=['CHANNEL','CITY'], inplace=True)

In [24]:
df_test['COUNTRY'] = label_encoder.transform(df_test['COUNTRY'])

In [26]:
X = df_test[['DELIVERY_REGION_ID','PRODUCT_ID', 'UNIT_PRICE','MONTH','YEAR','COUNTRY']]

In [27]:
pred = rf_model.predict(X)

In [28]:
denormalized_values = scaling.inverse_transform(pred.reshape(-1, 1))

In [29]:
denormalized_values

array([[8.21],
       [8.74],
       [8.39]])